In [1]:
import numpy as np
import pandas as pd
import uncertainties as unc
from uncertainties import unumpy

In [2]:
toxins = ['TacT3', 'AtaT', 'AtaT2', 'ItaT']
columns = ['aa'] + ['%s_Fmoc/Ac_%s' % (x, y) for x in toxins for y in ['Mean', 'SD']]

In [3]:
data_rrf = pd.read_excel('LC-MS_standart protocol.xlsx', sheet_name='Control_45_min', index_col=0, usecols=['aa', 'RRF_Mean', 'RRF_SD'])
data_5 = pd.read_excel('LC-MS_standart protocol.xlsx', sheet_name='Toxins_5_min', index_col=0, usecols=columns)
data_45 = pd.read_excel('LC-MS_standart protocol.xlsx', sheet_name='Toxins_45_min', index_col=0, usecols=columns)

In [4]:
rrf = unumpy.uarray(data_rrf['RRF_Mean'], data_rrf['RRF_SD'])
blank_df = pd.DataFrame(index=data_5.index)
frac_of_ac = {'5': {'Mean' : blank_df.copy(), 'SD' : blank_df.copy()}, '45': {'Mean' : blank_df.copy(), 'SD' : blank_df.copy()}}
for toxin in toxins:
    fmoc_ac_ratio_5 = unumpy.uarray(data_5['%s_Fmoc/Ac_Mean' % toxin], data_5['%s_Fmoc/Ac_SD' % toxin])
    fmoc_ac_ratio_45 = unumpy.uarray(data_45['%s_Fmoc/Ac_Mean' % toxin], data_45['%s_Fmoc/Ac_SD' % toxin])
    frac_of_ac_5 = 1/(1+rrf*fmoc_ac_ratio_5)
    frac_of_ac_45 = 1/(1+rrf*fmoc_ac_ratio_45)
    frac_of_ac['5']['Mean'][toxin] = unumpy.nominal_values(frac_of_ac_5)
    frac_of_ac['5']['SD'][toxin] = unumpy.std_devs(frac_of_ac_5)
    frac_of_ac['45']['Mean'][toxin] = unumpy.nominal_values(frac_of_ac_45)
    frac_of_ac['45']['SD'][toxin] = unumpy.std_devs(frac_of_ac_45)
frac_of_ac['5']['Mean'] = frac_of_ac['5']['Mean'].replace(1,0)
frac_of_ac['5']['SD'] = frac_of_ac['5']['SD'].replace(1,0)
frac_of_ac['45']['Mean'] = frac_of_ac['45']['Mean'].replace(1,0)
frac_of_ac['45']['SD'] = frac_of_ac['45']['SD'].replace(1,0)

In [5]:
with pd.ExcelWriter('fraction_of_acetylated_aa-tRNA.xlsx') as writer:
    for x in frac_of_ac:
        for y in frac_of_ac[x]:
            frac_of_ac[x][y].to_excel(writer, sheet_name='%s min %s' % (x, y))